In [ ]:
import torch
from torch.utils.data import Dataset
import random
import pandas as pd
from sentence_transformers import InputExample

# ==========================================
# 1. DATASET CLASSES
# ==========================================
class NexusTripletDataset(Dataset):
    """
    NEXUS v4.2: Original_ID bazlı Causal Triplet Mining.
    Modelin 'Aynı varlık, aynı olay, zıt sonuç' arasındaki farkı 
    (semantic difference) öğrenmesini sağlar.
    """
    def __init__(self, df):
        self.samples = []
        self.df = df
        
        # 1. Veriyi original_id bazında grupla
        # Bu, her haberin varyasyonlarını ve karşıtlarını bir arada tutar.
        self.grouped = df.groupby('original_id')
        
        # 2. Triplet oluşturma döngüsü
        for oid, group in self.grouped:
            # Anchor adayları (Orijinal veya augmented fark etmez)
            long_samples = group[group['label'] == 2]
            short_samples = group[group['label'] == 1]
            
            # --- LONG Anchor Senaryosu ---
            for _, anchor_row in long_samples.iterrows():
                # Positive: Aynı original_id içindeki diğer LONG haber (Augmented versiyon)
                # Eğer başka LONG yoksa, global havuzdan aynı asset'e sahip başka bir LONG çek
                pos_candidates = long_samples[long_samples.index != anchor_row.name]
                
                if not pos_candidates.empty:
                    positive = pos_candidates.sample(1).iloc[0]['text']
                else:
                    # Alternatif: Aynı asset'e sahip rastgele bir LONG haber (Dataset geneli)
                    asset = anchor_row['text'].split('[C]')[1] if '[C]' in anchor_row['text'] else ""
                    alt_pos = df[(df['label'] == 2) & (df['text'].str.contains(asset)) & (df.index != anchor_row.name)]
                    if not alt_pos.empty:
                        positive = alt_pos.sample(1).iloc[0]['text']
                    else: continue

                # HARD NEGATIVE: İşte zurnanın zırt dediği yer.
                # Aynı original_id içindeki SHORT haber (Counterfactual)
                # Model 'Bu haber neden LONG değil de SHORT?' sorusuna burada yanıt bulur.
                if not short_samples.empty:
                    negative = short_samples.sample(1).iloc[0]['text']
                    self.samples.append((anchor_row['text'], positive, negative))
                
            # --- SHORT Anchor Senaryosu ---
            for _, anchor_row in short_samples.iterrows():
                pos_candidates = short_samples[short_samples.index != anchor_row.name]
                
                if not pos_candidates.empty:
                    positive = pos_candidates.sample(1).iloc[0]['text']
                else:
                    asset = anchor_row['text'].split('[C]')[1] if '[C]' in anchor_row['text'] else ""
                    alt_pos = df[(df['label'] == 1) & (df['text'].str.contains(asset)) & (df.index != anchor_row.name)]
                    if not alt_pos.empty:
                        positive = alt_pos.sample(1).iloc[0]['text']
                    else: continue

                # HARD NEGATIVE: Aynı original_id içindeki LONG haber (Counterfactual)
                if not long_samples.empty:
                    negative = long_samples.sample(1).iloc[0]['text']
                    self.samples.append((anchor_row['text'], positive, negative))

        print(f"✓ Created {len(self.samples)} Causal Triplets from {len(self.grouped)} unique news events.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        anchor, pos, neg = self.samples[idx]
        return InputExample(texts=[anchor, pos, neg])

# ✅ DOĞRU (yeni)
class NexusDatasetV2(Dataset):
    def __init__(self, df):
        self.texts = df['text'].tolist()
        self.gates = (df['label'] != 0).astype(float).values  # S EKLE
        self.directions = np.where(df['label'] == 0, -1, (df['label'] == 2).astype(float))  # S EKLE
        self.tps = df['tp_pct'].fillna(0.0).values  # S EKLE
        self.validities = df['validity_minutes'].fillna(0.0).values  # S EKLE

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return (
            self.texts[idx],
            torch.tensor(self.gates[idx], dtype=torch.float32),
            torch.tensor(self.directions[idx],  dtype=torch.float32),
            torch.tensor(self.tps[idx],   dtype=torch.float32),
            torch.tensor(self.validities[idx],  dtype=torch.float32),
        )



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from scipy.optimize import minimize
from sklearn.metrics import log_loss

def calibrate_model(model, val_loader, device):
    """
    Validation set üzerinde optimal 'Temperature' değerini bulur.
    """
    model.eval()
    all_logits = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            texts, gate_gt, _, _, _ = batch
            inputs = model.tokenizer(list(texts), return_tensors="pt", padding=True, truncation=True).to(device)
            outputs = model(inputs['input_ids'], inputs['attention_mask'])
            
            all_logits.append(outputs['gate'].cpu())
            all_labels.append(gate_gt.cpu())

    logits = torch.cat(all_logits).numpy()
    labels = torch.cat(all_labels).numpy()

    # Log-loss minimize eden T değerini bul
    def objective(T):
        scaled_probs = 1 / (1 + np.exp(-logits / T))
        return log_loss(labels, scaled_probs)

    res = minimize(objective, x0=[1.0], bounds=[(0.01, 10.0)], method='L-BFGS-B')
    optimal_t = res.x[0]
    
    print(f"✓ Optimal Temperature Found: {optimal_t:.4f}")
    return optimal_t
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.1):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if not self.training or self.drop_prob == 0:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor

class TokenAwareAttentionHead(nn.Module):
    """
    Token-level attention ile critical information extraction.
    Mean pooling'i öldür, selektif okumayı başlat.
    """
    def __init__(self, hidden_size=768, num_heads=8, dropout=0.2):
        super().__init__()
        self.hidden_size = hidden_size
        self.drop_path = DropPath(drop_prob=0.2)
        # Learnable queries: Her task için model "neye bakmalı" öğreniyor
        self.task_query = nn.Parameter(torch.randn(1, 1, hidden_size))

        # Multi-head attention
        self.mha = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )

        # Post-attention processing
        self.norm1 = nn.LayerNorm(hidden_size)
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Dropout(dropout)
        )
        self.norm2 = nn.LayerNorm(hidden_size)

        # Final projection
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, token_embeddings, attention_mask=None):
        """
        Args:
            token_embeddings: (batch, seq_len, hidden_size)
            attention_mask: (batch, seq_len) - padding mask
        Returns:
            output: (batch, 1)
            attn_weights: (batch, 1, seq_len) - interpretability için
        """
        batch_size = token_embeddings.size(0)

        # Query'yi batch'e broadcast et
        query = self.task_query.expand(batch_size, -1, -1)
        # Key-value attention mask oluştur
        if attention_mask is not None:
            # attention_mask: (batch, seq_len) -> (batch, 1, seq_len)
            key_padding_mask = ~attention_mask.bool()
        else:
            key_padding_mask = None

        # Multi-head attention
        # query: (batch, 1, hidden) attends to
        # key/value: (batch, seq_len, hidden)
        attended, attn_weights = self.mha(
            query=query,
            key=token_embeddings,
            value=token_embeddings,
            key_padding_mask=key_padding_mask,
            need_weights=True
        )

        # Residual + LayerNorm
        x = self.norm1(query + self.drop_path(attended))

        # FFN
        x = self.norm2(x + self.ffn(x))

        # Final output
        output = self.output(x.squeeze(1))  # (batch, 1)

        return output, attn_weights

import torch
import torch.nn as nn
import torch.nn.functional as F

class DeepReasoningBlock(nn.Module):
    """
    NEXUS v4.5: Stacked Transformer Decoder Layers.
    Sadece pooling değil, adım adım muhakeme yapar.
    """
    def __init__(self, hidden_size, num_layers=2, num_heads=8, dropout=0.2):
        super().__init__()
        # 1. Learnable Task Query (Karar vektörü)
        self.query = nn.Parameter(torch.randn(1, 1, hidden_size))
        
        # 2. Transformer Decoder Katmanları
        # Not: num_layers=2 finansal nedensellik için 'sweet spot' noktasıdır.
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.reasoning_tower = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
    def forward(self, token_embeddings, mask=None):
        batch_size = token_embeddings.size(0)
        query = self.query.expand(batch_size, -1, -1)
        
        # Padding mask'i Transformer formatına (float mask) çeviriyoruz
        if mask is not None:
            # 0 (padding) -> -inf, 1 (valid) -> 0
            key_padding_mask = (mask == 0)
        else:
            key_padding_mask = None
            
        # Reasoning Tower: Query, tokenlar üzerinde birden fazla kez 'düşünür'
        # tgt: Query, memory: Encoder (DeBERTa) çıktıları
        x = self.reasoning_tower(
            tgt=query,
            memory=token_embeddings,
            memory_key_padding_mask=key_padding_mask
        )
        
        return x.squeeze(1) # (batch, hidden_size)

from transformers import AutoModel, AutoTokenizer

class NexusV2Production(nn.Module):
    def __init__(self, backbone_name="microsoft/deberta-v3-base"):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(backbone_name)
        self.tokenizer = AutoTokenizer.from_pretrained(backbone_name)
        hidden_size = self.encoder.config.hidden_size

        # DERİN BLOKLAR (num_layers=2)
        # Sadece Pooling değil, hiyerarşik muhakeme.
        self.gate_reasoner = DeepReasoningBlock(hidden_size, num_layers=2)
        self.dir_reasoner = DeepReasoningBlock(hidden_size, num_layers=2)

        # ÇIKIŞ KATMANLARI
        self.gate_out = nn.Linear(hidden_size, 1)
        self.dir_out = nn.Linear(hidden_size, 1)

        # Regresyon artık Gate'in 'düşünülmüş' context'inden besleniyor.
        self.tp_head = nn.Sequential(nn.Linear(hidden_size, 128), nn.GELU(), nn.Linear(128, 1), nn.Tanh())
        self.val_head = nn.Sequential(nn.Linear(hidden_size, 128), nn.GELU(), nn.Linear(128, 1), nn.Tanh())

    def forward(self, input_ids, attention_mask, return_attentions=False):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = encoder_outputs.last_hidden_state

        # Muhakeme Süreci
        # Artık bir liste olarak attention ağırlıklarını da alabilirsin.
        gate_context = self.gate_reasoner(token_embeddings, attention_mask)
        dir_context = self.dir_reasoner(token_embeddings, attention_mask)

        # Karar ve Regresyon
        gate_logit = self.gate_out(gate_context)
        dir_logit = self.dir_out(dir_context)
        tp_out = self.tp_head(gate_context) * 5.0 # Max %5 TP
        val_out = self.val_head(gate_context) * 20.0 # Max 20dk

        return {
            'gate': gate_logit,
            'direction': dir_logit,
            'tp': tp_out,
            'validity': val_out
        }

    def predict(self, texts, threshold=0.5, temperature=1.0):
        """
        NEXUS v4.5 Inference: Derin muhakeme sonuçlarını parse eder.
        """
        self.eval()
        device = next(self.parameters()).device
        
        if isinstance(texts, str):
            texts = [texts]

        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        ).to(device)

        with torch.no_grad():
            outputs = self.forward(
                input_ids=inputs['input_ids'], 
                attention_mask=inputs['attention_mask']
            )

            gate_probs = torch.sigmoid(outputs['gate'] / temperature).cpu().numpy()
            dir_probs = torch.sigmoid(outputs['direction'] / temperature).cpu().numpy()
            tp_preds = outputs['tp'].cpu().numpy()
            val_preds = outputs['validity'].cpu().numpy()

        results = []
        for i in range(len(texts)):
            g_prob = gate_probs[i][0]
            d_prob = dir_probs[i][0]
            
            # Aksiyon Kararı
            gate_decision = 1 if g_prob >= threshold else 0
            
            # Güven Skoru Normalizasyonu
            dir_conf = d_prob if d_prob >= 0.5 else (1 - d_prob)
            
            res = {
                'text': texts[i],
                'gate': gate_decision,
                'gate_confidence': round(float(g_prob), 4),
                'direction': 'LONG' if d_prob >= 0.5 else 'SHORT',
                'direction_confidence': round(float(dir_conf), 4),
                # Sadece aksiyon varsa değer ata, yoksa 0.
                'take_profit_pct': round(float(tp_preds[i][0]), 2) if gate_decision == 1 else 0.0,
                'validity_minutes': round(float(val_preds[i][0]), 1) if gate_decision == 1 else 0.0,
            }
            results.append(res)

        return results if len(results) > 1 else results[0]

In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, mean_absolute_error, mean_squared_error
from transformers import get_linear_schedule_with_warmup
from peft import LoraConfig, get_peft_model, TaskType
import random
from tqdm import tqdm
# ==========================================
# TRAINING CONFIGURATION
# ==========================================
class TrainingConfig:
    # Stage 1 options
    USE_LORA_STAGE1 = True  # False for Option B
    TRIPLET_REASONING_RATIO = 1  # 0.7 = hybrid, 1.0 = full reasoning

    # Stage 2 options
    FREEZE_LORA_STAGE2 = True  # True for Option A

    # Model selection
    BACKBONE_MODEL = "microsoft/DeBERTa-v3-base"  # Don't change to medium/large yet!

    # Data
    AUGMENTATION_MAX_PER_ORIGINAL = 8
    OOD_TEST_RATIO = 0.05

    # Training
    STAGE1_EPOCHS = 3
    STAGE2_EPOCHS = 5
    BATCH_SIZE = 16

    # Thresholds
    OOD_F1_THRESHOLD = 0.70
    HOLD_PRECISION_THRESHOLD = 0.85
    CAUSALITY_THRESHOLD = 0.80

config = TrainingConfig()


# ==========================================
# 2. LORA CONFIGURATION - GPT FIXED
# ==========================================
def apply_lora_final(st_model, r=16):
    """
    GPT Fix: output.dense ÇIKARILDI
    Sadece attention + intermediate.dense
    """
    target_model = st_model._first_module().auto_model

    lora_config = LoraConfig(
        r=r,
        lora_alpha=r*2,
        target_modules=[
            "query_proj",
            "key_proj",
            "value_proj",
            "intermediate.dense"
            # output.dense ÇIKARILDI - representation collapse riski
        ],
        lora_dropout=0.3,
        bias="none",
        task_type=TaskType.FEATURE_EXTRACTION
    )

    st_model._first_module().auto_model = get_peft_model(target_model, lora_config)
    st_model._first_module().auto_model.print_trainable_parameters()
    return st_model


# ==========================================
# 3. STAGE 1: BACKBONE TRAINING
# ==========================================
class CausalTripletTrainer:
    def __init__(self, model, train_loader, margin=0.3, lr_backbone=1e-5, lr_reasoner=1e-4):
        self.model = model
        self.train_loader = train_loader
        self.device = next(model.parameters()).device
        
        # Triplet Loss: Cosine Distance tabanlı
        self.criterion = nn.TripletMarginWithDistanceLoss(
            distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y),
            margin=margin
        )
        
        # Diferansiyel Learning Rate: Reasoner 'Hot', LoRA 'Warm'
        self.optimizer = AdamW([
            {'params': [p for n, p in model.encoder.named_parameters() if "lora_" in n], 'lr': lr_backbone},
            {'params': model.gate_reasoner.parameters(), 'lr': lr_reasoner}
        ], weight_decay=0.01)

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        pbar = tqdm(self.train_loader, desc="Stage 1: Causal Triplet Training")
        
        for batch in pbar:
            # Batch: (anchor, positive, negative)
            anchors, positives, negatives = batch
            
            self.optimizer.zero_grad()
            
            # 1. Embeddings: Her üç haber de gate_reasoner'dan geçip 'Context Vector' üretir.
            # Bu vektör, modelin o haber hakkındaki 'rafine düşüncesidir'.
            a_emb = self._get_embedding(anchors)
            p_emb = self._get_embedding(positives)
            n_emb = self._get_embedding(negatives)
            
            # 2. Loss Calculation
            loss = self.criterion(a_emb, p_emb, n_emb)
            
            # 3. Optimization
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0) # Gradyan patlaması koruması
            self.optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'triplet_loss': f'{loss.item():.4f}'})
            
        return total_loss / len(self.train_loader)

    def _get_embedding(self, texts):
        """Haber metnini alıp gate_reasoner çıkışındaki 768d context vektörünü döner."""
        inputs = self.model.tokenizer(
            list(texts), return_tensors="pt", padding=True, truncation=True, max_length=256
        ).to(self.device)
        
        # Forward pass: Sadece context vektörünü alıyoruz (logitleri değil)
        encoder_out = self.model.encoder(inputs['input_ids'], inputs['attention_mask']).last_hidden_state
        context, _ = self.model.gate_reasoner(encoder_out, inputs['attention_mask'])
        return context

def get_nexus_optimizer(model, base_lr=1e-4, lora_lr=1e-5):
    """
    NEXUS Differential Learning Rate Strategy.
    1. Backbone (DeBERTa): Frozen (0 LR)
    2. LoRA Layers: Warm (1e-5) - Hizalama için esnek bırakıldı.
    3. Reasoning Blocks: Hot (1e-4) - Yeni mantığı bunlar öğrenecek.
    4. Task Heads: Hot (1e-4)
    """
    no_decay = ["bias", "LayerNorm.weight"]
    
    optimizer_grouped_parameters = [
        # Grup 1: Yeni eklenen Reasoning Blocklar ve Headler (Yüksek LR)
        {
            "params": [p for n, p in model.named_parameters() 
                       if ("reasoner" in n or "out" in n or "head" in n) 
                       and not any(nd in n for nd in no_decay)],
            "weight_decay": 0.05,
            "lr": base_lr,
        },
        # Grup 2: LoRA Adaptörleri (Düşük LR - Backbone'u bozmadan esnet)
        {
            "params": [p for n, p in model.named_parameters() 
                       if "lora_" in n and not any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": lora_lr,
        },
        # Bias ve LayerNormlar için Weight Decay kapatma
        {
            "params": [p for n, p in model.named_parameters() 
                       if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": base_lr,
        },
    ]

    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, eps=1e-8)
    return optimizer


def setup_scheduler(optimizer, train_loader, epochs=8):
    total_steps = len(train_loader) * epochs
    # Warmup oranını %15'e çıkarıyoruz çünkü yeni bloklarımız çok 'ham'
    warmup_steps = int(total_steps * 0.15)
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    return scheduler

# ==========================================
# 4. COMBINED LOSS - GPT FIXED
# ==========================================
def nexus_combined_loss(outputs, targets, gamma=2.0, label_smoothing=0.1):
    """
    NEXUS v7_Final: DeepReasoningBlock ile tam uyumlu.
    Verbalizer yükü atıldı, tüm baskı 'Directional Causality' üzerine kuruldu.
    """
    gate_p = outputs['gate']
    dir_p = outputs['direction']
    tp_p = outputs['tp']
    val_p = outputs['validity']

    gate_gt, dir_gt, tp_gt, val_gt = targets

    # 1. GATE LOSS: Kalibre edilmiş BCE
    # Aksiyon sinyallerini (2.0 weight) 'Hold' gürültüsünden ayırır.
    gate_gt_smoothed = gate_gt * (1 - label_smoothing) + 0.5 * label_smoothing
    loss_gate = F.binary_cross_entropy_with_logits(
        gate_p, gate_gt_smoothed, pos_weight=torch.tensor([2.0]).to(gate_p.device)
    )

    # 2. DIRECTIONAL FOCAL LOSS (En Kritik Bölge)
    # Verbalizer yok, bu yüzden modelin tek rehberi burası.
    valid_dir_mask = (dir_gt >= 0).squeeze()
    if valid_dir_mask.sum() > 0:
        # Bullish Bias Killer: SHORT (0) hala 3 kat daha maliyetli.
        dir_weights = torch.where(dir_gt[valid_dir_mask] == 0, 3.0, 1.0)
        
        probs = torch.sigmoid(dir_p[valid_dir_mask])
        pt = torch.where(dir_gt[valid_dir_mask] == 1, probs, 1 - probs)
        focal_weight = (1 - pt) ** gamma # Zor örnekleri (zıt haberler) daha sert cezalandır.
        
        bce_dir = F.binary_cross_entropy_with_logits(
            dir_p[valid_dir_mask], 
            dir_gt[valid_dir_mask], 
            reduction='none'
        )
        loss_dir = (dir_weights * focal_weight * bce_dir).mean()
    else:
        loss_dir = torch.tensor(0.0, device=gate_p.device)

    # 3. REGRESSION LOSS: Robust Huber
    # TP ve Validity artık 'Düşünülmüş Context'ten geliyor.
    action_mask = (gate_gt == 1).squeeze()
    if action_mask.sum() > 0:
        loss_tp = F.huber_loss(tp_p[action_mask], tp_gt[action_mask], delta=1.0)
        loss_val = F.huber_loss(val_p[action_mask], val_gt[action_mask], delta=1.0)
    else:
        loss_tp = loss_val = torch.tensor(0.0, device=gate_p.device)

    # TOPLAM KAYIP: Yön doğruluğu (Reasoning) ana öncelik.
    # Dir (4.0) > Gate (1.0) > Regression (0.5)
    total_loss = (1.0 * loss_gate) + (4.0 * loss_dir) + (0.5 * (loss_tp + loss_val))

    return total_loss, {
        'gate': loss_gate.item(),
        'dir_focal': loss_dir.item(),
        'tp_huber': loss_tp.item(),
        'val_huber': loss_val.item()
    }

# ==========================================
# 6. OUT-OF-DISTRIBUTION TEST
# ==========================================
def create_ood_test_set(df, ood_ratio=0.05):
    """Her class'tan %5'ini çek"""
    print("\n" + "="*50)
    print("CREATING OUT-OF-DISTRIBUTION TEST SET")
    print("="*50)

    long_df = df[df['label'] == 2]
    short_df = df[df['label'] == 1]
    hold_df = df[df['label'] == 0]

    ood_long = long_df.sample(frac=ood_ratio, random_state=42)
    ood_short = short_df.sample(frac=ood_ratio, random_state=42)
    ood_hold = hold_df.sample(frac=ood_ratio, random_state=42)

    ood_test = pd.concat([ood_long, ood_short, ood_hold]).reset_index(drop=True)
    train_df = df[~df.index.isin(ood_test.index)].reset_index(drop=True)

    print(f"OOD Test Set: {len(ood_test)} samples")
    print(f"  - Long: {len(ood_long)} | Short: {len(ood_short)} | Hold: {len(ood_hold)}")
    print(f"Remaining Train: {len(train_df)} samples")

    return train_df, ood_test





# ==========================================
# 7. COUNTERFACTUAL TEST
# ==========================================
def counterfactual_test(model):
    """Causal reasoning testi"""
    print("\n" + "="*50)
    print("COUNTERFACTUAL CAUSALITY TEST")
    print("="*50)

    test_pairs = [
        # Basic flip (mevcut)
        ("[N] Bitcoin ETF approved by SEC [C] BTC",
        "[N] Bitcoin ETF rejected by SEC [C] BTC"),
        ("[N] Ethereum upgrade successful [C] ETH",
        "[N] Ethereum upgrade delayed [C] ETH"),

        # Numeric reversal
        ("[N] Solana gains 15% in morning trading session [C] SOL",
        "[N] Solana drops 15% in morning trading session [C] SOL"),

        # Subtle negation
        ("[N] Concerns over Polygon network stability prove unfounded [C] MATIC",
        "[N] Concerns over Polygon network stability prove justified [C] MATIC"),

        # Implicit sentiment
        ("[N] Bitcoin finally breaks resistance at $70K after months [C] BTC",
        "[N] Bitcoin fails again to break resistance at $70K [C] BTC"),

        # Multi-entity (tricky)
        ("[N] Ethereum outperforms Bitcoin in Q4 rally [C] ETH",
        "[N] Bitcoin outperforms Ethereum in Q4 rally [C] BTC"),

        # Sarcastic/Ironic (advanced)
        ("[N] Experts say crypto winter is 'definitely' over this time [C] BTC",
        "[N] Experts confirm crypto winter continues with no end in sight [C] BTC"),
    ]

    passed = 0
    total = len(test_pairs)

    for orig, counter in test_pairs:
        orig_pred = model.predict(orig, temperature=optimal_t)
        counter_pred = model.predict(counter, temperature=optimal_t)

        print(f"\n{'─'*50}")
        print(f"Original: {orig}")
        print(f"  → Gate: {orig_pred['gate']} | Dir: {orig_pred['direction']} | TP: {orig_pred['take_profit_pct']:.2f}%")
        print(f"Counterfactual: {counter}")
        print(f"  → Gate: {counter_pred['gate']} | Dir: {counter_pred['direction']} | TP: {counter_pred['take_profit_pct']:.2f}%")

        if orig_pred['direction'] != counter_pred['direction']:
            print("  ✓ PASS: Direction flipped correctly")
            passed += 1
        else:
            print("  ✗ FAIL: Direction didn't flip")

    print(f"\n{'='*50}")
    print(f"Counterfactual Test Result: {passed}/{total} passed ({passed/total*100:.1f}%)")
    print(f"{'='*50}")

    return passed / total


# ==========================================
# 8. MAIN TRAINING PIPELINE - GPT FIXED
# ==========================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# YENİ EKLEME - Kullanıcı uyarısı
print("\n" + "="*50)
print("MODEL SELECTION: DeBERTa-v3-SMALL")
print("="*50)
print("⚠️  Medium/Large model kullanmayın!")
print("Gerekli metrikler için minimum threshold:")
print("  - OOD Gate F1 ≥ 0.70")
print("  - HOLD Precision ≥ 0.85")
print("  - Counterfactual Flip ≥ 80%")
print("Bu threshold'lar aşılmadan model büyütme = overfit!")
print("="*50 + "\n")
# Load data
print("\nLoading data...")
df = pd.read_json('/content/nexus_elite_dataset.json')
print(f"Total samples: {len(df)}")
print(f"Label distribution:\n{df['label'].value_counts()}")
# Augmentation limit
print("\nApplying augmentation limit...")

    # OOD split
def create_true_ood_split_with_report(df, ood_ratio=0.05):
    """ID bazlı split yapar ve benzersiz haber dağılımını raporlar."""
    unique_ids = df['original_id'].unique()
    np.random.shuffle(unique_ids)

    split_idx = int(len(unique_ids) * ood_ratio)
    ood_ids = unique_ids[:split_idx]

    train_df = df[~df['original_id'].isin(ood_ids)].reset_index(drop=True)
    ood_df = df[df['original_id'].isin(ood_ids)].reset_index(drop=True)

    def get_id_stats(target_df, name):
        # Her label için benzersiz ID'leri say
        stats = target_df.groupby('label')['original_id'].nunique()
        total_unique = target_df['original_id'].nunique()
        print(f"\n📊 {name} Seti Benzersiz Haber Analizi (Unique IDs):")
        print(f"   - Toplam Benzersiz: {total_unique}")
        print(f"   - HOLD  (0): {stats.get(0, 0)} unique news")
        print(f"   - SHORT (1): {stats.get(1, 0)} unique news")
        print(f"   - LONG  (2): {stats.get(2, 0)} unique news")
        return stats

    # Raporları yazdır
    get_id_stats(train_df, "TRAIN + VAL")
    get_id_stats(ood_df, "OOD TEST")

    return train_df, ood_df

# Ana akışta bunu kullan:
train_df, ood_test_df = create_true_ood_split_with_report(df, ood_ratio=0.05)

# Train/Val split
gss = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
train_idx, val_idx = next(gss.split(train_df, groups=train_df['original_id']))
train_df_final = train_df.iloc[train_idx].reset_index(drop=True)
val_df = train_df.iloc[val_idx].reset_index(drop=True)

print(f"Train: {len(train_df_final)} | Val: {len(val_df)} | OOD Test: {len(ood_test_df)}")

# STAGE 1: Triplet Training
trainer = CausalTripletTrainer(model, triplet_loader, margin=0.3)
for epoch in range(config.STAGE1_EPOCHS):
    avg_loss = trainer.train_epoch()
    print(f"Epoch {epoch} Triplet Loss: {avg_loss}")

Using device: cuda

MODEL SELECTION: DeBERTa-v3-SMALL
⚠️  Medium/Large model kullanmayın!
Gerekli metrikler için minimum threshold:
  - OOD Gate F1 ≥ 0.70
  - HOLD Precision ≥ 0.85
  - Counterfactual Flip ≥ 80%
Bu threshold'lar aşılmadan model büyütme = overfit!


Loading data...
Total samples: 11646
Label distribution:
label
0    8308
2    1877
1    1461
Name: count, dtype: int64

Applying augmentation limit...

📊 TRAIN + VAL Seti Benzersiz Haber Analizi (Unique IDs):
   - Toplam Benzersiz: 8084
   - HOLD  (0): 7892 unique news
   - SHORT (1): 85 unique news
   - LONG  (2): 107 unique news

📊 OOD TEST Seti Benzersiz Haber Analizi (Unique IDs):
   - Toplam Benzersiz: 425
   - HOLD  (0): 416 unique news
   - SHORT (1): 6 unique news
   - LONG  (2): 3 unique news
Train: 9283 | Val: 1770 | OOD Test: 593

STAGE 1: BACKBONE TRAINING WITH TRIPLET LOSS


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


trainable params: 1,622,016 || all params: 185,453,568 || trainable%: 0.8746
Triplet strategy: 70% reasoning, 30% self-contrast

Training on 2205 triplets for 3 epochs...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss



✓ Backbone training complete. Saved to 'nexus-lora-backbone-final'


In [ ]:

import os
import re
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, mean_absolute_error, mean_squared_error
from transformers import get_linear_schedule_with_warmup
from peft import LoraConfig, get_peft_model, TaskType
import random
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def create_true_ood_split_with_report(df, ood_ratio=0.05):
    """ID bazlı split yapar ve benzersiz haber dağılımını raporlar."""
    unique_ids = df['original_id'].unique()
    np.random.shuffle(unique_ids)

    split_idx = int(len(unique_ids) * ood_ratio)
    ood_ids = unique_ids[:split_idx]

    train_df = df[~df['original_id'].isin(ood_ids)].reset_index(drop=True)
    ood_df = df[df['original_id'].isin(ood_ids)].reset_index(drop=True)
    def get_id_stats(target_df, name):
        # Her label için benzersiz ID'leri say
        stats = target_df.groupby('label')['original_id'].nunique()
        total_unique = target_df['original_id'].nunique()
        print(f"\n📊 {name} Seti Benzersiz Haber Analizi (Unique IDs):")
        print(f"   - Toplam Benzersiz: {total_unique}")
        print(f"   - HOLD  (0): {stats.get(0, 0)} unique news")
        print(f"   - SHORT (1): {stats.get(1, 0)} unique news")
        print(f"   - LONG  (2): {stats.get(2, 0)} unique news")
        return stats
    # Raporları yazdır
    get_id_stats(train_df, "TRAIN + VAL")
    get_id_stats(ood_df, "OOD TEST")

    return train_df, ood_df

# Ana akışta bunu kullan:
train_df, ood_test_df = create_true_ood_split_with_report(df, ood_ratio=0.05)
# Train/Val split
gss = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
train_idx, val_idx = next(gss.split(train_df, groups=train_df['original_id']))
train_df_final = train_df.iloc[train_idx].reset_index(drop=True)
val_df = train_df.iloc[val_idx].reset_index(drop=True)
print(f"Train: {len(train_df_final)} | Val: {len(val_df)} | OOD Test: {len(ood_test_df)}")

model = NexusV2Production("nexus-lora-backbone-final").to(device)  # ← Stage 1'den yükle
optimizer = get_nexus_optimizer(model)
scheduler = setup_scheduler(optimizer, train_loader)
# Stage 2 LoRA freeze
if config.FREEZE_LORA_STAGE2:
    print("\n⚠️  OPTION A: Freezing LoRA in Stage 2")
    for name, param in model.encoder.named_parameters():
        param.requires_grad = False
else:
    for name, param in model.named_parameters():
        if "lora_" in name:
            param.requires_grad = True
        elif "classifier" in name or "regressor" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False
# Stage 2: Multi-head training
print("\n" + "="*50)
print("STAGE 2: MULTI-HEAD TRAINING")
print("="*50)
# GPT Fix: POS_WEIGHT KALDIRILDI
print("\n⚠️  Using WeightedRandomSampler (pos_weight=1.0)")
gate_pos_weight = torch.tensor([4.0], dtype=torch.float32, device=device)  # Kullanılmayacak
# Weighted sampler
train_labels = (train_df_final['label'] != 0).astype(int).values
class_counts = np.bincount(train_labels)
# ✅ DOĞRU (yeni)
class_weights = np.sqrt(1.0 / class_counts)
sample_weights = class_weights[train_labels]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True,
)
print(f"Class balance: Hold={class_counts[0]}, Action={class_counts[1]}")
print(f"Sampler weights: Hold={class_weights[0]:.3f}, Action={class_weights[1]:.3f}")
# Datasets
train_dataset = NexusDatasetV2(train_df_final)
val_dataset = NexusDatasetV2(val_df)
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
# Model
# Freeze non-LoRA
for name, param in model.encoder.named_parameters():
    if "lora" not in name:
        param.requires_grad = False
# ✅ EKLE - LoRA'yı da freeze et (Option A)
print("\n⚠️  OPTION A: Freezing LoRA weights in Stage 2")
for name, param in model.encoder.named_parameters():
    # Tüm encoder'ı freeze et (LoRA dahil)
    param.requires_grad = False
print("✓ All encoder weights frozen. Only task heads will be trained.")
# Scheduler
num_training_steps = len(train_loader) * 5
# GPT Fix: Early stopping SADECE val_loss
best_val_loss = float('inf')
patience_counter = 0
max_epochs = 12
optimal_t = calibrate_model(model, val_loader, device)
for epoch in range(max_epochs):
    # TRAIN
    model.train()
    train_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{max_epochs} [TRAIN]")
    for texts, gate_gt, dir_gt, tp_gt, val_gt in pbar:
        optimizer.zero_grad()
        gate_gt = gate_gt.view(-1, 1).to(device)
        dir_gt = dir_gt.view(-1, 1).to(device)
        tp_gt = tp_gt.view(-1, 1).to(device)
        val_gt = val_gt.view(-1, 1).to(device)
        inputs = model.tokenizer(list(texts), return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
        preds = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], gate_labels=gate_gt)
        loss, _ = nexus_combined_loss(preds, (gate_gt, dir_gt, tp_gt, val_gt))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
    avg_train_loss = train_loss / len(train_loader)
    # VALIDATION
    model.eval()
    val_loss = 0
    all_gate_preds = []
    all_gate_gts = []
    all_dir_preds = []
    all_dir_gts = []
    all_tp_preds = []
    all_tp_gts = []
    all_val_preds = []
    all_val_gts = []
    with torch.no_grad():
        for texts, gate_gt, dir_gt, tp_gt, val_gt in tqdm(val_loader, desc=f"Epoch {epoch+1}/5 [VAL]"):
            gate_gt = gate_gt.view(-1, 1).to(device)
            dir_gt = dir_gt.view(-1, 1).to(device)
            tp_gt = tp_gt.view(-1, 1).to(device)
            val_gt = val_gt.view(-1, 1).to(device)
            inputs = model.tokenizer(list(texts), return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
            preds = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], gate_labels=gate_gt)
            loss, _ = nexus_combined_loss(preds, (gate_gt, dir_gt, tp_gt, val_gt))
            val_loss += loss.item()
            gate_p = preds['gate']
            dir_p = preds['direction']
            tp_p = preds['tp']
            val_p = preds['validity']
            gate_pred = (torch.sigmoid(gate_p) > 0.5).float()
            all_gate_preds.extend(gate_pred.cpu().numpy())
            all_gate_gts.extend(gate_gt.cpu().numpy())
            action_mask = (gate_gt == 1).squeeze()
            valid_dir_mask = (dir_gt >= 0).squeeze()  # -1 olan HOLD'ları filtrele
            combined_mask = action_mask & valid_dir_mask
            if combined_mask.sum() > 0:
                dir_pred = (torch.sigmoid(dir_p[combined_mask]) > 0.5).float()
                all_dir_preds.extend(dir_pred.cpu().numpy())
                all_dir_gts.extend(dir_gt[combined_mask].cpu().numpy())
                all_tp_preds.extend(tp_p[action_mask].cpu().numpy())
                all_tp_gts.extend(tp_gt[action_mask].cpu().numpy())
                all_val_preds.extend(val_p[action_mask].cpu().numpy())
                all_val_gts.extend(val_gt[action_mask].cpu().numpy())
    avg_val_loss = val_loss / len(val_loader)
    # Metrics
    gate_acc = accuracy_score(all_gate_gts, all_gate_preds)
    gate_prec, gate_rec, gate_f1, _ = precision_recall_fscore_support(
        all_gate_gts, all_gate_preds, average='binary', zero_division=0
    )
    print(f"\n{'='*50}")
    print(f"Epoch {epoch+1}/5 Results")
    print(f"{'='*50}")
    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"Gate - Acc: {gate_acc:.3f} | Prec: {gate_prec:.3f} | Rec: {gate_rec:.3f} | F1: {gate_f1:.3f}")
    if len(all_dir_preds) > 0:
        dir_acc = accuracy_score(all_dir_gts, all_dir_preds)
        tp_mae = mean_absolute_error(all_tp_gts, all_tp_preds)
        val_mae = mean_absolute_error(all_val_gts, all_val_preds)
        print(f"Direction Acc: {dir_acc:.3f}")
        print(f"TP MAE: {tp_mae:.3}")
        print(f"Validity MAE: {val_mae:.3}")
# Early stopping - SADECE val_loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "nexus_best.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss,
            'val_f1': gate_f1
        }, "nexus_checkpoint.pt")
        print(f"✓ Model saved (val_loss: {avg_val_loss:.4f})")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"✗ No improvement ({patience_counter}/3)")
        if patience_counter >= 3:
            print("Early stopping triggered!")
            break


Using device: cuda

MODEL SELECTION: DeBERTa-v3-SMALL
⚠️  Medium/Large model kullanmayın!
Gerekli metrikler için minimum threshold:
  - OOD Gate F1 ≥ 0.70
  - HOLD Precision ≥ 0.85
  - Counterfactual Flip ≥ 80%
Bu threshold'lar aşılmadan model büyütme = overfit!


Loading data...
Total samples: 11646
Label distribution:
label
0    8308
2    1877
1    1461
Name: count, dtype: int64

Applying augmentation limit...

📊 TRAIN + VAL Seti Benzersiz Haber Analizi (Unique IDs):
   - Toplam Benzersiz: 8084
   - HOLD  (0): 7897 unique news
   - SHORT (1): 84 unique news
   - LONG  (2): 103 unique news

📊 OOD TEST Seti Benzersiz Haber Analizi (Unique IDs):
   - Toplam Benzersiz: 425
   - HOLD  (0): 411 unique news
   - SHORT (1): 7 unique news
   - LONG  (2): 7 unique news
Train: 9240 | Val: 1770 | OOD Test: 636

⚠️  OPTION A: Freezing LoRA in Stage 2

STAGE 2: MULTI-HEAD TRAINING

⚠️  Using WeightedRandomSampler (pos_weight=1.0)
Class balance: Hold=6720, Action=2520
Sampler weights: Hold=0.012, A

Epoch 1/5 [VAL]: 100%|██████████| 111/111 [00:34<00:00,  3.18it/s]



Epoch 1/5 Results
Train Loss: 27.5011 | Val Loss: 17.7543
Gate - Acc: 0.985 | Prec: 0.995 | Rec: 0.960 | F1: 0.977
Direction Acc: 0.715
TP MAE: 2.27
Validity MAE: 4.27
✓ Model saved (val_loss: 17.7543)


Epoch 2/5 [VAL]: 100%|██████████| 111/111 [00:35<00:00,  3.17it/s]



Epoch 2/5 Results
Train Loss: 18.3536 | Val Loss: 17.6702
Gate - Acc: 0.989 | Prec: 0.997 | Rec: 0.971 | F1: 0.984
Direction Acc: 0.740
TP MAE: 2.2
Validity MAE: 4.29
✓ Model saved (val_loss: 17.6702)


...


Epoch 4/5 Results
Train Loss: 17.5005 | Val Loss: 17.8626
Gate - Acc: 0.991 | Prec: 0.995 | Rec: 0.978 | F1: 0.986
Direction Acc: 0.776
TP MAE: 2.24
Validity MAE: 4.13
✗ No improvement (1/3)


Epoch 5/5 [VAL]: 100%|██████████| 111/111 [00:35<00:00,  3.14it/s]



Epoch 5/5 Results
Train Loss: 17.3882 | Val Loss: 17.6634
Gate - Acc: 0.991 | Prec: 0.995 | Rec: 0.978 | F1: 0.986
Direction Acc: 0.767
TP MAE: 2.21
Validity MAE: 4.23
✗ No improvement (2/3)


Epoch 6/5 [VAL]: 100%|██████████| 111/111 [00:34<00:00,  3.18it/s]


Epoch 6/5 Results
Train Loss: 17.2890 | Val Loss: 17.6634
Gate - Acc: 0.991 | Prec: 0.995 | Rec: 0.978 | F1: 0.986
Direction Acc: 0.767
TP MAE: 2.21
Validity MAE: 4.23
✗ No improvement (3/3)
Early stopping triggered!


In [ ]:
def evaluate_ood(model, ood_df, device):
    """OOD test set evaluation"""
    print("\n" + "="*50)
    print("OUT-OF-DISTRIBUTION EVALUATION")
    print("="*50)

    model.eval()
    ood_dataset = NexusDatasetV2(ood_df)
    ood_loader = DataLoader(ood_dataset, batch_size=16, shuffle=False)

    all_gate_preds = []
    all_gate_gts = []
    all_dir_preds = []
    all_dir_gts = []
    all_tp_preds = []
    all_tp_gts = []
    all_val_preds = []
    all_val_gts = []

    with torch.no_grad():
        for texts, gate_gt, dir_gt, tp_gt, val_gt in ood_loader:
            gate_gt = gate_gt.view(-1, 1).to(device)
            dir_gt = dir_gt.view(-1, 1).to(device)
            tp_gt = tp_gt.view(-1, 1).to(device)
            val_gt = val_gt.view(-1, 1).to(device)

            inputs = model.tokenizer(list(texts), return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
            preds = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], gate_labels=gate_gt)
            loss, _ = nexus_combined_loss(preds, (gate_gt, dir_gt, tp_gt, val_gt))
            val_loss += loss.item()
            gate_p = preds['gate']
            dir_p = preds['direction']
            tp_p = preds['tp']
            val_p = preds['validity']

            gate_pred = (torch.sigmoid(gate_p) > 0.5).float()
            all_gate_preds.extend(gate_pred.cpu().numpy())
            all_gate_gts.extend(gate_gt.cpu().numpy())

            action_mask = (gate_gt == 1).squeeze()
            valid_dir_mask = (dir_gt >= 0).squeeze()  # -1 olan HOLD'ları filtrele
            combined_mask = action_mask & valid_dir_mask
            if combined_mask.sum() > 0:
                dir_pred = (torch.sigmoid(dir_p[combined_mask]) > 0.5).float()
                all_dir_preds.extend(dir_pred.cpu().numpy())
                all_dir_gts.extend(dir_gt[combined_mask].cpu().numpy())

                all_tp_preds.extend(tp_p[action_mask].cpu().numpy())
                all_tp_gts.extend(tp_gt[action_mask].cpu().numpy())

                all_val_preds.extend(val_p[action_mask].cpu().numpy())
                all_val_gts.extend(val_gt[action_mask].cpu().numpy())

    # Metrics
    gate_acc = accuracy_score(all_gate_gts, all_gate_preds)
    gate_prec, gate_rec, gate_f1, _ = precision_recall_fscore_support(
        all_gate_gts, all_gate_preds, average='binary', zero_division=0
    )

    print(f"\nGate Performance:")
    print(f"  Accuracy: {gate_acc:.3f}")
    print(f"  Precision: {gate_prec:.3f}")
    print(f"  Recall: {gate_rec:.3f}")
    print(f"  F1: {gate_f1:.3f}")

    dir_acc = 0.0
    tp_mae = 0.0
    val_mae = 0.0

    if len(all_dir_preds) > 0:
        dir_acc = accuracy_score(all_dir_gts, all_dir_preds)
        print(f"\nDirection Performance (on actions):")
        print(f"  Accuracy: {dir_acc:.3f}")

        tp_mae = mean_absolute_error(all_tp_gts, all_tp_preds)
        tp_rmse = np.sqrt(mean_squared_error(all_tp_gts, all_tp_preds))

        val_mae = mean_absolute_error(all_val_gts, all_val_preds)
        val_rmse = np.sqrt(mean_squared_error(all_val_gts, all_val_preds))

        print(f"\nRegression Performance:")
        print(f"  TP - MAE: {tp_mae:.3f}% | RMSE: {tp_rmse:.3f}%")
        print(f"  Validity - MAE: {val_mae:.3f}min | RMSE: {val_rmse:.3f}min")

    return {
        'gate_acc': gate_acc,
        'gate_f1': gate_f1,
        'gate_precision': gate_prec,  # GPT: HOLD precision için
        'dir_acc': dir_acc,
        'tp_mae': tp_mae,
        'val_mae': val_mae
    }

# ===== LOAD BEST MODEL =====
print("\n" + "="*50)
print("LOADING BEST MODEL")
print("="*50)
model = NexusV2Production("/content/drive/MyDrive/Nexus-AI-Models/Sentence-Transformers/Model1/nexus-lora-backbone-final")
model.load_state_dict(torch.load("/content/drive/MyDrive/Nexus-AI-Models/Sentence-Transformers/Model1/nexus_best.pt"))
print("✓ Best model loaded")
# ===== COUNTERFACTUAL TEST =====
causality_score = counterfactual_test(model)
# ===== OOD EVALUATION =====
ood_metrics = evaluate_ood(model, ood_test_df, device)
# ===== FINAL REPORT =====
print("\n" + "="*50)
print("FINAL TRAINING REPORT")
print("="*50)
print(f"Best Validation Loss: {best_val_loss:.4f}")
# EKLEME - HOLD precision kontrolü
print(f"\n⚠️  CRITICAL METRICS CHECK:")
print(f"  OOD Gate F1: {ood_metrics['gate_f1']:.3f} (Target: ≥0.70)")
print(f"  HOLD Precision: {ood_metrics['gate_precision']:.3f} (Target: ≥0.85)")
print(f"  Causality Score: {causality_score*100:.1f}% (Target: ≥80%)")
if ood_metrics['gate_f1'] < 0.70:
    print("\n❌ WARNING: Gate F1 too low. More real data needed!")
if ood_metrics['gate_precision'] < 0.85:
    print("❌ WARNING: HOLD Precision too low. Will overtrade in production!")
if causality_score < 0.80:
    print("❌ WARNING: Causality test failed. Model not learning causal patterns!")
print("\n⚠️  ARCHITECTURE RISKS:")
if ood_metrics['gate_f1'] < 0.65:
    print("  - Low OOD F1: Consider hybrid triplet strategy (reasoning + self-contrast)")
print("  - Current triplet uses reasoning as positive (LLM style bias risk)")
print("  - Monitor performance on Telegram/Twitter news without reasoning")
print(f"\nOOD Test Performance:")
print(f"  Gate Accuracy: {ood_metrics['gate_acc']:.3f}")
print(f"  Direction Accuracy: {ood_metrics['dir_acc']:.3f}")
print(f"  TP MAE: {ood_metrics['tp_mae']:.3f}%")
print(f"  Validity MAE: {ood_metrics['val_mae']:.3f}min")
# ===== SAVE FINAL MODEL =====
print("\n" + "="*50)
print("SAVING FINAL MODEL")
print("="*50)
# Full model save
model.encoder.save("nexus-final-encoder")
torch.save({
    'gate_head': model.gate_head.state_dict(),
    'dir_head': model.dir_head.state_dict(),
    'tp_head': model.tp_head.state_dict(),
    'val_head': model.val_head.state_dict(),
    'config': {
        'hidden_size': model.encoder.get_sentence_embedding_dimension(),
        'best_val_loss': best_val_loss,
        'causality_score': causality_score,
        'ood_metrics': ood_metrics
    }
}, "nexus-final-heads.pt")
print("✓ Final model saved to 'nexus-final-encoder' and 'nexus-final-heads.pt'")


LOADING BEST MODEL
✓ Best model loaded

COUNTERFACTUAL CAUSALITY TEST

──────────────────────────────────────────────────
Original: [N] Bitcoin ETF approved by SEC [C] BTC
  → Gate: 0 | Dir: LONG | TP: 0.99%
Counterfactual: [N] Bitcoin ETF rejected by SEC [C] BTC
  → Gate: 0 | Dir: SHORT | TP: 0.89%
  ✓ PASS: Direction flipped correctly

──────────────────────────────────────────────────
Original: [N] Ethereum upgrade successful [C] ETH
  → Gate: 0 | Dir: LONG | TP: 1.21%
Counterfactual: [N] Ethereum upgrade delayed [C] ETH
  → Gate: 0 | Dir: SHORT | TP: 1.21%
  ✓ PASS: Direction flipped correctly

──────────────────────────────────────────────────
Original: [N] Solana gains 15% in morning trading session [C] SOL
  → Gate: 1 | Dir: LONG | TP: 0.95%
Counterfactual: [N] Solana drops 15% in morning trading session [C] SOL
  → Gate: 1 | Dir: LONG | TP: 0.87%
  ✗ FAIL: Direction didn't flip

──────────────────────────────────────────────────
Original: [N] Concerns over Polygon network stab

RuntimeError: Expected all tensors to be on the same device, but got index is on cuda:0, different from other tensors on cpu (when checking argument in method wrapper_CUDA__index_select)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil # Import shutil for cross-filesystem moves

source_file = 'nexus_best.pt'
destination_path = '/content/drive/MyDrive/Nexus-AI-Models/Sentence-Transformers/Model1'

if os.path.exists(source_file):
    os.makedirs(destination_path, exist_ok=True)
    # Use shutil.move instead of os.rename for cross-device links
    shutil.move(source_file, os.path.join(destination_path, source_file))
    print(f"'{source_file}' successfully moved to '{destination_path}'")
else:
    print(f"Error: '{source_file}' not found in the current directory.")

'nexus_best.pt' successfully moved to '/content/drive/MyDrive/Nexus-AI-Models/Sentence-Transformers/Model1'


In [ ]:
import os
import shutil

# Taşımak istediğiniz klasörün yolu
source_folder = '/content/nexus-lora-backbone-final'

# Google Drive'ınızdaki hedef yol
# Örn: '/content/drive/MyDrive/YeniKlasörüm'
destination_path = '/content/drive/MyDrive/Nexus-AI-Models/Sentence-Transformers/Model1'

# Klasörün varlığını kontrol et
if os.path.exists(source_folder):
    # Hedef yolu oluştur (eğer yoksa)
    os.makedirs(destination_path, exist_ok=True)

    # Klasörü taşı
    # shutil.move, farklı dosya sistemleri arasında bile çalışır
    shutil.move(source_folder, os.path.join(destination_path, os.path.basename(source_folder)))
    print(f"'{source_folder}' klasörü başarıyla '{destination_path}' konumuna taşındı.")
else:
    print(f"Hata: '{source_folder}' mevcut dizinde bulunamadı.")

'/content/nexus-lora-backbone-final' klasörü başarıyla '/content/drive/MyDrive/Nexus-AI-Models/Sentence-Transformers/Model1' konumuna taşındı.
